In [1]:
from orion_recommend.datasets import DatasetMap, synthetic, utils
from orion.sources import S3Source
from orion.sources.io import read_csv, write_csv
from orion.sources import RedshiftSource
import pandas as pd
import numpy as np
from scipy import sparse

In [2]:
#source = RedshiftSource(query='SELECT * FROM publish.inventory_lookup WHERE productid IS NULL')
#df_stock = source.read_csv()

with S3Source(key="masters/uploads/customers/1560425511130_Peak_customers.csv", bucket="kilimanjaro-prod-datalake") as s3:
    df_cust = read_csv(s3)

df_trans = read_csv(S3Source(key="masters/uploads/transactions/1560426066385_Peak_transactions.csv", bucket="kilimanjaro-prod-datalake"))
df_prod = read_csv(S3Source(key="masters/uploads/product/1560425499995_Peak_product.csv", bucket="kilimanjaro-prod-datalake"))


/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# 30pc of transactions have keys that are not in df_cust
pd.Series(df_trans.customerkey.unique()).isin(df_cust.customerkey).sum()/len(df_trans.customerkey.unique())
# 45pc of item numbers not in df_prod
pd.Series(df_trans.itemnumber.unique()).isin(df_prod.itemnumber).sum()/len(df_trans.itemnumber.unique())

0.5487752814984791

In [4]:
# Fix item number 
df_prod.itemnumber = pd.to_numeric(df_prod.itemnumber, errors='coerce') 

In [5]:
# Drop nas
df_prod.dropna(subset=["itemnumber"], inplace=True)

In [6]:
# Remove keys not in customer and product tables
df_trans = df_trans.iloc[df_trans.customerkey[df_trans.customerkey.isin(df_cust.customerkey)].index,:]
df_trans.reset_index(inplace=True, drop=True)
df_trans = df_trans.iloc[df_trans.itemnumber[df_trans.itemnumber.isin(df_prod.itemnumber)].index,:]

# Reset index before changing keys to integers
df_trans.reset_index(inplace=True, drop=True)
df_prod.reset_index(inplace=True, drop=True)


In [7]:
df_prod.dropna()

,itemnumber,brandcode,category,colourvalue,divisioncode,fabric,itemcategorycode,itemcolour,itemfamilycode,itemquarter,itemseason,itemstylecode,productgroup
1,83567.0,FEK,Mens,BLUE,APPAREL,Unknown,AM,Blue,WB,Quarter 2,SS,83566,T-shirts
6,89686.0,SU,Mens,BLACK,APPAREL,Unknown,AM,Black,DM,Quarter 1,SS,89686,Hoods
9,116110.0,PP,Womens,BLACK,FOOTWEAR,Unknown,FW,White/White,FA,Quarter 3,AW,116107,Basketball
11,92233.0,NN,Mens,BLACK,FOOTWEAR,Unknown,FM,Black,FA,Quarter 3,AW,90975,Basketball
13,131524.0,RR,Womens,WHITE,FOOTWEAR,Unknown,FW,White,FA,Quarter 4,AW,131524,Nylon Running
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145428,126411.0,PP,Womens,WHITE,FOOTWEAR,Unknown,FW,White/Metallic Gold,FA,Quarter 4,AW,126411.0,Nylon Running
145432,82952.0,VV,Womens,WHITE,FOOTWEAR,Unknown,FW,White,FA,Quarter 3,AW,82952.0,Vulcanised
145433,111261.0,NAP,Mens,BLACK,APPAREL,Unknown,AM,Black,DM,Quarter 4,AW,111261.0,Jackets
145435,4031961.0,VV,Womens,RED,APPAREL,Unknown,AW,Chilli Pepper,FA,Quarter 4,AW,4028768.0,T-shirts


In [8]:
# Drop redundant cols. Fabric all unknown, item colour less info than colourvalue
df_prod.drop(["itemstylecode", "itemquarter", "itemcolour", "fabric"], axis=1, inplace=True)

In [9]:
# Drop columns
df_trans.drop(["salestransactionkey", "salesordernumber","discountpercent", "grosssales",
              'orderdate', 'ordertime', 'shippingdate','grossprofit'], axis=1, inplace=True)

In [10]:
df = df_trans.merge(df_cust, on='customerkey', how = "outer")
df = df.merge(df_prod, on = "itemnumber")

In [11]:
# Rename and drop columns, nas
df.rename(columns={"customerkey":"user_id","itemnumber":"item_id","shipcountry":"country", "brandcode":"brand_id",
                  "colourvalue":"colour", }, inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [12]:
df_trans.rename(columns={"customerkey":"user_id","itemnumber":"item_id"}, inplace=True)
df_cust.rename(columns={"customerkey":"user_id"}, inplace=True)
df_prod.rename(columns={"itemnumber":"item_id"}, inplace=True)

In [13]:
df_trans.drop_duplicates(subset=["user_id", "item_id", 'pricetype'], inplace=True)


In [14]:
cats = df.select_dtypes('object').columns
keys = df[cats].apply(lambda x: x.factorize()[1])
df[cats] = df[cats].apply(lambda x: x.factorize()[0])

In [15]:
user_num = len(df.user_id.unique())
item_num = len(df.item_id.unique())

In [16]:
# Renumber customer keys
ints = [i for i in range(user_num)]
cust_keys = [i for i in df.user_id.unique()]
customer_key_dict = {i:j for i,j in zip( cust_keys, ints)}

#Renumber item ids
ints = [i for i in range(item_num)]
item_keys = [i for i in df.item_id.unique()]
item_key_dict = {i:j for i,j in zip( item_keys, ints)}

In [17]:
# Change keys to integers in dataframes
df.item_id = [item_key_dict[df.item_id[i]] for i in range(len(df))]
df.user_id = [customer_key_dict[df.user_id[i]] for i in range(len(df))]

In [27]:
df.drop('unitssold', axis=1, inplace=True)

In [28]:
# Train-test split
train = df.sample(frac = 0.8)
ind = df.index.isin(train.index)
test = df[~ind]

In [30]:
np.save("fa_train_data",train.to_numpy())
np.save("fa_test_data",test.to_numpy())

In [19]:
USER_NUM = len(df.user_id.unique())
country_num = len(df.country.unique())
postcode_num = len(df.postcode.unique())	
item_id_num = len(df.item_id.unique())
pricetype_num = len(df.pricetype.unique())
unitssold_num = len(df.unitssold.unique())
loyaltyaccount_num = len(df.loyaltyaccount.unique())
gender_num = len(df.gender.unique())
brand_id_num = len(df.brand_id.unique())
category_num = len(df.category.unique())
colour_num = len(df.colour.unique())
divisioncode_num = len(df.divisioncode.unique())
itemcategorycode_num = len(df.itemcategorycode.unique())
itemfamilycode_num = len(df.itemfamilycode.unique())
itemseason_num = len(df.itemseason.unique())
productgroup_num = len(df.productgroup.unique())

### Item Attribute matrix

In [20]:
features = [  pricetype_num, brand_id_num, category_num, colour_num, divisioncode_num, 
            itemcategorycode_num, itemfamilycode_num, itemseason_num, productgroup_num]
names = [ 'pricetype', 'brand_id',
         'category', 'colour', 'divisioncode', 'itemcategorycode',
         'itemfamilycode', 'itemseason', 'productgroup']
dic = dict(zip(names,features))

In [25]:
dic.values()

dict_values([2, 335, 8, 59, 3, 13, 7, 3, 78])

In [31]:
matrices = []
for i in dic.keys():
    mat = np.zeros((item_num, dic[i]), dtype = np.int)    
    pair = df.loc[:,["item_id", i]]
    M = np.unique(pair, axis=0)
    for m in M:
        mat[m[0], m[1]] = 1
    print(mat[1])
    matrices.append(mat)

[1 1]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0]
[0 1 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [32]:
fa_matrix = np.concatenate((matrices), axis=1)

In [33]:
np.save('fa_ia_matrix.npy', fa_matrix)

In [73]:
matrices[0].shape

(37130, 2)

### User- attribute matrix

In [21]:
matrices = []
for i in dic.keys():
    mat = np.zeros((USER_NUM, dic[i]), dtype = np.int)
    pair = df.loc[:,["user_id", i]]
    M = np.unique(pair, axis=0)
    for m in M:
        mat[m[0], m[1]] = 1
    print(mat[1])
    matrices.append(mat)

[1 1]
[1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
[1 1 0 0 0 0 0 0]
[1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 0]
[1 1 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0]
[1 1 0]
[1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [22]:
fa_matrix = np.concatenate((matrices), axis=1)

In [23]:
np.save('fa_ua_matrix.npy', fa_matrix)